# Kode for datafortelling om operative mål i PO Pensjon
Målingene legges manuelt inn i et excel dokument som må lastes opp som "../data/Måleparametere PO Pensjon 2022.csv" ved oppdatering.

In [ ]:
import sys
import importlib
import pandas as pd
import numpy as np
import plotly
import plotly.express as px
import plotly.graph_objects as go
from typing import Dict

import datastory as ds

sys.path.append('../lib')
import operative as opera
importlib.reload(opera)

In [ ]:
df = pd.read_csv("../data/Måleparametere PO Pensjon 2022.csv", sep=";", header=2)

In [ ]:
df["prosent_flagg"] = df.Nullpunktsmåling.apply(lambda x: "%" in x)

In [ ]:
# Propager operative mål:
tmp = df["Operativt mål"].isna()
for i in range(df.shape[0]):
    if tmp[i]:
        df.iloc[i, 1] = df.iloc[i-1, 1]

In [ ]:
def clean_numbers(x):
    if "%" in x:
        tmp = x[:-2]
        x = str(float(tmp)/100)
    if "-" in x:
        x = "0"
    return "".join(x.split(" "))

In [ ]:
def set_plus(x):
    if x[0].isnumeric():
        return "+" + x
    elif x.strip() == "-":
        return "+0 %"
    else:
        return x

In [ ]:
# Få alle tall på riktig format
maalinger_cols = ['Nullpunktsmåling', 'Måling T1'] # Legg til nye målinger her
for col in maalinger_cols:
    df[col] = df[col].apply(lambda x: clean_numbers(x)).astype("float")

prosent_endring_cols = ['% endring mot NP'] # Legg til ny endringsprosent hver tertial
for col in prosent_endring_cols:
    df[col] = df[col].apply(lambda x: set_plus(x))

In [ ]:
df.tail()

In [ ]:
# Hardkoder en linje fordi NP er "-" og ikke "0%":
df.loc[df.shape[0]-1, "prosent_flagg"] = True     

### Fjern målepunkter med bare "-" (hvis ønskelig):
#df.loc[df.shape[0]-1, "Nullpunktsmåling"] = np.nan
#df.drop([0,7,11,17,18,19,28] , inplace=True)

In [ ]:
df = df.T

In [ ]:
df

In [ ]:
# Liste med operative mål
operatives = []
prev_op = ""
for i in range(df.shape[1]):
    # Hver kolonne er ett målepunkt
    col = df.iloc[:,i]
    op = col["Operativt mål"]
    mparam = col["Måleparameter"]
    new_op = op != prev_op
    if new_op:
        print(op)
        # Append objekt for opprettet mål
        if prev_op != "":
            operatives.append(Op)
        # Lag objekt for neste operative mål
        Op = opera.Operative(op)
    prev_op = op
    # Oppretter målepunkts-objekt for kolonnen
    Op.MPs[mparam] = opera.MP.from_series(col=col, n=len(operatives)%7)
# Append siste operative mål
operatives.append(Op)

In [ ]:
#operatives[1].show_figs()

In [ ]:
story = ds.DataStory("Operative mål i PO Pensjon")
story.markdown("Oversikt over operative mål i PO Pensjon og tilhørende målepunkter. Enkelte målepunkter er ikke inkludert her. Prosent i plottene angir endring fra nullpunktsmåling.")
for op in operatives:
    story.header(op.name, level=1)
    print(op.name)
    for _, mp in op.MPs.items():
        story.header(mp.name, level=2)
        if not pd.isna(mp.col.Beskrivelse):
            story.markdown(mp.col.Beskrivelse)
        story.plotly(mp.fig.to_json())
        if not pd.isna(mp.col.Kommentar):
            story.markdown("*Kommentar:* " + mp.col.Kommentar)
        print(mp.name)

In [ ]:
#story.publish(url="https://nada.ekstern.dev.nav.no/api") #dev
story.publish(url="https://nada.intern.nav.no/api") #prod
#story.update(url="https://nada.intern.nav.no/api", token='ed32f304-8fc4-489f-b08d-c6b1b719f22e')